# Replay & Fork state

## State History using SDK

### Redefine financial advicer graph with intent checker (from 11_dynamic-breakpoints.ipynb)

In [ ]:
import requests
import yfinance as yf
from pprint import pformat
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import MessagesState, START, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode
from langgraph.checkpoint.memory import MemorySaver
from IPython.display import Image, display
from langgraph.errors import NodeInterrupt


# Defining Tools
##################################################################################

def lookup_stock_symbol(company_name: str) -> str:
    """
    Converts a company name to its stock symbol using a financial API.

    Parameters:
        company_name (str): The full company name (e.g., 'Tesla').

    Returns:
        str: The stock symbol (e.g., 'TSLA') or an error message.
    """
    api_url = "https://www.alphavantage.co/query"
    params = {
        "function": "SYMBOL_SEARCH",
        "keywords": company_name,
        "apikey": "your_alphavantage_api_key_2"
    }
    
    response = requests.get(api_url, params=params)
    data = response.json()
    
    if "bestMatches" in data and data["bestMatches"]:
        return data["bestMatches"][0]["1. symbol"]
    else:
        return f"Symbol not found for {company_name}."


def fetch_stock_data_raw(stock_symbol: str) -> dict:
    """
    Fetches comprehensive stock data for a given symbol and returns it as a combined dictionary.

    Parameters:
        stock_symbol (str): The stock ticker symbol (e.g., 'TSLA').
        period (str): The period to analyze (e.g., '1mo', '3mo', '1y').

    Returns:
        dict: A dictionary combining general stock info and historical market data.
    """
    period = "1mo"
    try:
        stock = yf.Ticker(stock_symbol)

        # Retrieve general stock info and historical market data
        stock_info = stock.info  # Basic company and stock data
        stock_history = stock.history(period=period).to_dict()  # Historical OHLCV data

        # Combine both into a single dictionary
        combined_data = {
            "stock_symbol": stock_symbol,
            "info": stock_info,
            "history": stock_history
        }

        return pformat(combined_data)

    except Exception as e:
        return {"error": f"Error fetching stock data for {stock_symbol}: {str(e)}"}


# Binding tools to the LLM
##################################################################################

# Create tool bindings with additional attributes
lookup_stock = Tool.from_function(
    func=lookup_stock_symbol,
    name="lookup_stock_symbol",
    description="Converts a company name to its stock symbol using a financial API.",
    return_direct=False  # Return result to be processed by LLM
)

fetch_stock = Tool.from_function(
    func=fetch_stock_data_raw,
    name="fetch_stock_data_raw",
    description="Fetches comprehensive stock data including general info and historical market data for a given stock symbol.",
    return_direct=False
)

toolbox = [lookup_stock, fetch_stock]

# OPENAI_API_KEY environment variable must be set
simple_llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = simple_llm.bind_tools(toolbox)


# Defining Agent's node
##################################################################################

# System message
assistant_system_message = SystemMessage(content=("""
You are a professional financial assistant specializing in stock market analysis and investment strategies. 
Your role is to analyze stock data and provide **clear, decisive recommendations** that users can act on, 
whether they already hold the stock or are considering investing.

You have access to a set of tools that can provide the data you need to analyze stocks effectively. 
Use these tools to gather relevant information such as stock symbols, current prices, historical trends, 
and key financial indicators. Your goal is to leverage these resources efficiently to generate accurate, 
actionable insights for the user.

Your responses should be:
- **Concise and direct**, summarizing only the most critical insights.
- **Actionable**, offering clear guidance on whether to buy, sell, hold, or wait for better opportunities.
- **Context-aware**, considering both current holders and potential investors.
- **Free of speculation**, relying solely on factual data and trends.
- **do not forget** to provide stock name in the report, so it's clear which stock is being recommended.

### Response Format:
1. **Recommendation:** Buy, Sell, Hold, or Wait.
2. **Key Insights:** Highlight critical trends and market factors that influence the decision.
3. **Suggested Next Steps:** What the user should do based on their current position.

If the user does not specify whether they own the stock, provide recommendations for both potential buyers and current holders. Ensure your advice considers valuation, trends, and market sentiment.

Your goal is to help users make informed financial decisions quickly and confidently.
"""))

# Node
def assistant(state: MessagesState):
   return {"messages": [llm_with_tools.invoke([assistant_system_message] + state["messages"])]}

def intent_check(state: MessagesState):
    user_request = state["messages"][-1].content

    financial_check_prompt = f"""
    You are an intent classifier. Your task is to determine if the user's request is specifically related to finance, investments, or financial advice.

    Evaluate the following user request:

    "{user_request}"

    If the request is about finance, investments, or financial advice, respond with "True".
    If it is unrelated to finance, respond with "False".

    Respond with only "True" or "False" and nothing else.
    """

    llm_response = llm_with_tools.invoke([HumanMessage(content=financial_check_prompt)]).content
    is_financial_question = llm_response.strip().lower() == 'true'

    if not is_financial_question:
        raise NodeInterrupt("Please ask a question related to financial advice.")

    return state


# Defining Graph
##################################################################################
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("intent_check", intent_check)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(toolbox))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "intent_check")
builder.add_edge("intent_check", "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "assistant")

memory = MemorySaver()
graph = builder.compile(checkpointer=memory)
display(Image(graph.get_graph().draw_mermaid_png()))

### Execute the graph and interrupt it

In [ ]:
# start a new conversation
thread = {"configurable": {"thread_id": "1"}}

# define intiial user request
initial_input = {"messages": HumanMessage(content="What is the weather outside?")}

# run the graph and stream in values mode
for event in graph.stream(initial_input, thread, stream_mode="values"):
    event['messages'][-1].pretty_print()

### Return current checkpoint

In [ ]:
state = graph.get_state(thread)
print(state.next)
print("\n\n")
print(state)

### Get the most recent Chekpoint
![get-most-recent-checkpoint](images/get-most-recent-checkpoint.png)

### Let's finish the graph

In [ ]:
# start a new conversation
thread = {"configurable": {"thread_id": "2"}}

# define intiial user request
initial_input = {"messages": HumanMessage(content="What is a stock symbol for Tesla?")}

# run the graph and stream in values mode
for event in graph.stream(initial_input, thread, stream_mode="values"):
    event['messages'][-1].pretty_print()

In [ ]:
state = graph.get_state(thread)
print(state.next)
print("\n\n")
print(state)

### Multiple Checkpoints
![multiple-checkpoints](images/multiple-checkpoints.png)


In [ ]:
all_states = [s for s in graph.get_state_history(thread)]
len(all_states)

### Replay the Graph

Let's replay the graph from assistant.

In [ ]:
to_replay = all_states[-3]
to_replay.values

Check next Node to run

In [ ]:
to_replay.next

Check config with `checkpoint_id` and `thread_id`

In [ ]:
to_replay.config

and finally let's reply!

In [ ]:
for event in graph.stream(None, to_replay.config, stream_mode="values"):
    event['messages'][-1].pretty_print()

**Important!**

Replay does not actually trigger real execution!

### Forking with SDK
![forking](images/forking.png)

In [ ]:
to_fork = all_states[-3]
to_fork.values

In [ ]:
print(to_fork.next)
print(to_fork.config)

### Modify specific checkpoint

In [ ]:
fork_config = graph.update_state(to_fork.config, {
    "messages": [HumanMessage(content='What is a stock symbol for nvidia?', id=to_fork.values["messages"][0].id)]},
)
print(fork_config)

State was updated but next node still the same

In [ ]:
state = graph.get_state(thread)
print(state.next)
print("\n\n")
print(state)

### Execute the fork (reak work instead of simple replay)

In [ ]:
for event in graph.stream(None, fork_config, stream_mode="values"):
    event['messages'][-1].pretty_print()

In [ ]:
state = graph.get_state(thread)
print(state.next)

## State History using API

In [ ]:
from langgraph_sdk import get_client

URL = "http://localhost:57698"
client = get_client(url=URL)

assistants = await client.assistants.search()
assistants

### Replay with API

#### Run the graph

In [ ]:
from langchain_core.messages import convert_to_messages

thread = await client.threads.create()
input_message = HumanMessage(content="What is a stock symbol for Tesla?")

async for event in client.runs.stream(
            thread["thread_id"], 
            assistant_id="financial_advisor_intent_check",
            input={"messages": [input_message]}, 
            stream_mode="values",
):
    messages = event.data.get('messages', None)
    if messages:
        print(convert_to_messages(messages)[-1])
        print("\n\n")

#### Get the `checkpoint_id` from state

In [ ]:
states = await client.threads.get_history(thread['thread_id'])
print(len(states))
to_replay = states[-5]
to_replay

#### Replay the graph

In [ ]:
async for event in client.runs.stream(
            thread["thread_id"], 
            assistant_id="financial_advisor_intent_check",
            input=None, 
            stream_mode="values",
            checkpoint_id=to_replay['checkpoint_id']
):
    messages = event.data.get('messages', None)
    if messages:
        print(f"\n\n{convert_to_messages(messages)[-1]}")

### Forking with API

In [ ]:
to_fork = states[-3]
print(to_fork['values'])
print("\n\n")
print(to_fork['next'])
print("\n\n")
print(to_fork['checkpoint_id'])

#### Fork the checkpoint

In [ ]:
forked_config = await client.threads.update_state(
    thread["thread_id"],
    {"messages": HumanMessage(
        content="What is a stock symbol for nvidia?", 
        id=to_fork['values']['messages'][0]['id'])},
    checkpoint_id=to_fork['checkpoint_id']
)
print(forked_config)

#### Resume the graph with forked checkpoint

In [ ]:
async for event in client.runs.stream(
            thread["thread_id"], 
            assistant_id="financial_advisor_intent_check",
            input=None, 
            stream_mode="values",
            checkpoint_id=forked_config['checkpoint_id']
):
    messages = event.data.get('messages', None)
    if messages:
        print(f"\n\n{convert_to_messages(messages)[-1]}")